# Анализ рынка вакансий для HR агентства

**Цель проекта:**

Выявить признаки вакансий, привлекающих наибольшее внимание кандидатов на работу.

**Описание данных**

vacancy_clear.xlsx:
* id - уникальный идентификатор вакансии
* Статус - статус вакансии. Возможные значения:
     - Архив - вакансия закрыта, не удалось найти подходящую кандидатуру силами агентства;
     - Закрыта нами - вакансия закрыта, подходящая кандидатура найдена силами агентства;
     - Активна - вакансия опубликована, идет активный поиск кандидатов;
     - На рассмотрении - пул кандидатов набран, заказчик осуществляет отбор, набор новых кандидатов не ведется;
     - Отклонена - заказчик отказался от рассмотрения пула кандидатов, набор кандидатов не ведется;
     - Черновик - вакансия в стадии заполнения.
* Источник лида - источник вакансии. Возможные значения:
     - Входящая заявка - заказчик сам заполнил форму на сайте КА;
     - Сорсинг - менеджеры КА привлекли заказчика исходящим поиском;
     - Повторная - это не первая вакансия компании, подбор на которую будет делать КА;
     - Реферальная - заказчик пришел в КА по рекомендации от другого заказчика.
* Менеджер - менеджер, ответственный за вакансию
* Дата публикации - дата, когда вакансия была опубликована на сайте КА
* Дата закрытия - дата, когда заказчик одобрил кандидата КА или отказался от услуг КА по данной вакансии
* Количество просмотров - количество уникальных пользователей сайта, кликнувших на заголовок вакансии, чтобы увидеть всю вакансию
* Количество откликов - количество уникальных пользователей сайта, отправивших отклик на вакансию
* Позиция - название вакансии
* Зарплата от - нижняя граница заработной платы для вакансии
* Зарплата до - верхняя граница заработной платы для вакансии
* Город - в каких городах предлагается работа вне зависимости от формата
* Формат оформления - какой формат оформления предлагает заказчик
* Формат работы - какой формат работы предлагает заказчик
* Опыт - какой опыт работы должен быть у соискателя
* Образование - какое образование должно быть у соискателя
* Ссылка на тестовое - есть ли ссылка на тестовое задание в описании вакансии
* Занятость - какой формат занятости предлагает заказчик
* Обязательные требования - какие обязательные требования предъявляет заказчик к кандидатам
* Дополнительный требования - какие дополнительные требования предъявляет заказчик к кандидатам
* Этапы отбора - этапы отбора на должность
* Условия - краткое текстовое описание условий вакансии

**План работы**<a id='6'></a>
1. [Ознакомление с данными](#1) (изучение общей информации о датасетах)
* количество строк и столбцов
* количество и доля пропусков

2. [Предобработка данных](#2)
* изменение типов данных
* поиск и удаление явных дубликатов

3. [Исследовательский анализ данных](#3)
* изучить имеющиеся в датасете признаки и сформировать новые для проверки гипотез 

4. [Проверка гипотез](#4)
* проверка гипотез для выяснения признаков, влияющих на привлекательность вакансий для кандидатов

5. [Общий вывод](#5)
* подведение итогов исследования

<a id='1'></a>
## [Ознакомление с данными](#6)

In [131]:
import pandas as pd
import numpy as np
from plotly import graph_objects as go
from scipy import stats as st

In [132]:
#уберем ограничения на количество строк и столбцов в таблицах.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

Прочитаем файл с вакансиями.

In [133]:
url='https://docs.google.com/spreadsheets/d/135-Ti4pP0CdaiDQLQfhFkErP0NgxTwR1/edit?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
data = pd.read_excel(url)

Определим функцию для получения основной информации о датафрейме. Функция выведет первые 5 строк датафрейма, информацию о датафрейме с помощью метода df.info(), а также количество пропущенных значений в датафрейме и их долю.

In [134]:
def observe(df):
    import numpy as np
    display(df.head())
    print('')
    print(df.info())
    print('')
    display(
        pd.DataFrame(
            np.array([df.isna().sum(), df.isna().mean() * 100]).T,
            columns = ['кол-во пропусков', 'доля пропусков'],
            index=df.columns
        ).style.background_gradient('coolwarm').format({'кол-во пропусков' : '{:.0f}', 'доля пропусков' : '{:.2f}%'})
    )

In [135]:
observe(data)

,id,Статус,Источник лида,Менеджер,Дата публикации,Дата закрытия,Количество просмотров,Количество откликов,Позиция,Зарплата от,Зарплата до,Город,Формат оформления,Формат работы,Опыт,Образование,Занятость,Ссылка на тестовое,Обязательные требования,Дополнительный требования,Этапы отбора,Условия
0,40,Архив,Повторная,менеджер 9,2023-08-30,2023-08-30,153,35,Стажёр Python разработчик (неоплачиваемая стажировка для НКО),NaN,NaN,Москва,"ИП, ГПХ, Самозанятость, Контракт, Трудовой договор",Удаленка,"До 1 года опыта, возможен опыт в pet-проектах; без опыта","Другое, На смежных специальностях, Профильное","Проектная, Частичная",нет,"Наш стек: Python, fastapi, postgres, gitlab\nПожелания:\n- Опыт использования одного из фреймворков/библиотек для написания сервисов на RESTful API\n- Понимание как работает асинхронность и многопоточность в Python\n- Знание основ Git\n- Базовое знание SQL","Основные требования к ребятам, которые хотели бы пройти практику в школе IT - это:\n- 8+ месяцев обучения на соответсвующем направлении\n- Один или несколько законченных pet проектов\n\nОстальные требования - это больше пожелания, нежели твердые требования. \n\nЕсли человек закончил 8-12 месяцев обучения, то он скорее всего уже успел познакомиться с перечисленным выше стеком (на поверхностном уровне)",Техническое задание и интервью,Школа бесплатна и сотрудничает с фондами.\n\nСтажировка не оплачивается участникам.\n\nВ конце проекта ты сразу сможешь пополнить им своё портфолио. Проект будет реализован людьми для людей.
1,104,Архив,Входящая заявка,менеджер 6,2023-08-22,2023-08-22,188,4,Технический интервьюер,NaN,NaN,Москва,ГПХ,Удаленка,"До 1 года опыта, возможен опыт в pet-проектах; без опыта","Другое, На смежных специальностях, Профильное",Частичная,нет,"1) Широкий IT-кругозор, понимание, какие бывают языки программирования и для чего они используются.\n2) Владение хотя быть одним современным языком программирования (Java, Python, PHP, C#, Go, JavaScript и/или C++), желательно знание на базовом уровне языка запросов SQL и базовые навыки вёрстки веб-страниц (HTML/CSS).\n3) Свободное владение разговорным английским языком.\n4) Системность, аналитический склад ума и умение чувствовать других людей.\n5) Желание и готовность проводить собеседования и оценивать специалистов",NaN,1) тестовое задание\n2) онлайн собеседование с менеджером проектов\n3) онлайн собеседование с СОО или СЕО,ставка 750 рублей - за 1 обработанного кандидата (проведено собеседование + написано заключение по нему) \nПолная удаленка\nОбучение
2,157,Архив,Реферальная,менеджер 9,2023-08-28,2023-10-03,37,8,JS-разработчик,NaN,100000.0,Москва,"ИП, ГПХ, Самозанятость, Контракт, Трудовой договор",Удаленка,"До 1 года опыта, возможен опыт в pet-проектах","Другое, На смежных специальностях, Профильное",Полная,нет,"умение самостоятельно разбираться в чужом коде;_x000D_\nопыт самостоятельной работы над коммерческими проектами;_x000D_\nспособность к самоорганизации и самодисциплине;_x000D_\nумение оценивать трудоемкость задачи и планировать свое рабочее время;_x000D_\nнаходится на связи большую часть рабочего дня (время по МСК);_x000D_\nумение грамотно изложить свои вопросы/идеи._x000D_\n_x000D_\nопыт верстки (CSS 3, Flex и Sass);_x000D_\nхороший уровень знания Javascript и его асинхронных особенностей;_x000D_\nхорошее знание Vue.js v2 (понимание реактивных свойств, умение работать с компонентами);_x000D_\nопыт работы с Node JS и Express;_x000D_\nхорошее знание синтаксиса SQL, умение читать и писать не простые запросы (у нас PostgreSQL);_x000D_\nумение обсуждать ход проекта с другими участниками (используем Telegram, Zoom);_x000D_\nспособность внятно документировать свой код.",Опыт коммерческой разработки на JS будет большим преимуществом.,"- Отклик с сопроводительным письмом обязательно! Коротко рассказать о себе (навыки, заслуги, увлечения), о проектах, в которых довелось участвовать, о своей мотивации на компанию и задачи\n- Собеседование (обычно оно проходит в текстовом или голосовом режиме в Telegram


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         1284 non-null   int64  
 1   Статус                     1284 non-null   object 
 2   Источник лида              1274 non-null   object 
 3   Менеджер                   1193 non-null   object 
 4   Дата публикации            1284 non-null   object 
 5   Дата закрытия              1127 non-null   object 
 6   Количество просмотров      1284 non-null   int64  
 7   Количество откликов        1284 non-null   int64  
 8   Позиция                    1284 non-null   object 
 9   Зарплата от                469 non-null    float64
 10  Зарплата до                372 non-null    float64
 11  Город                      1284 non-null   object 
 12  Формат оформления          1284 non-null   object 
 13  Формат работы              1284 non-null   obje

,кол-во пропусков,доля пропусков
id,0,0.00%
Статус,0,0.00%
Источник лида,10,0.78%
Менеджер,91,7.09%
Дата публикации,0,0.00%
Дата закрытия,157,12.23%
Количество просмотров,0,0.00%
Количество откликов,0,0.00%
Позиция,0,0.00%
Зарплата от,815,63.47%


В датафрейме 1284 строк и 22 столбца. Имеется достаточное количетсво данных для проведения исследования. В датафрейме есть пропущеные значения которые заполняются опционально. Столбцы `Дата публикации` и `Дата закрытия` нужно привести в формат даты.

<a id='2'></a>
## [Предобработка данных](#6)

Переименуем столбцы в таблице, чтобы они начинались с маленькой буквы.

In [136]:
data.columns = data.columns.str.lower()

Изменим тип столбцов `дата публикации` и `дата закрытия` в датафрейме на datetime.

In [137]:
data['дата публикации'] = pd.to_datetime(data['дата публикации'])
data['дата закрытия'] = pd.to_datetime(data['дата закрытия'])

Проверим датафрейм на наличие явных дубликатов.

In [138]:
print('Количество явных дубликатов в датафрейме: {0}, доля явных дубликатов: {1:.2f}%'\
          .format(data.duplicated().sum(), data.duplicated().sum() / len(data) * 100))

Количество явных дубликатов в датафрейме: 0, доля явных дубликатов: 0.00%


Явные дубликаты в датафрейме отсутствуют.

На этапе предобработки данных мы переименовали столбцы в таблице, чтобы они начинались с маленькой буквы, изменили тип столбцов `дата публикации` и `дата закрытия` в датафрейме на datetime, убедились в отсутствии явных дубликатов в датафреймах.

<a id='3'></a>
## [Исследовательский анализ данных](#6)

### Анализ различных признаков у вакансий

Посмотрим количество вакансий по статусам.

In [139]:
status = data.groupby('статус').agg(count=('статус', 'count')).sort_values(by='count', ascending=False)\
.reset_index()
status['ratio_pct'] = round(status['count'] / len(data) * 100, 2)
status

,статус,count,ratio_pct
0,Архив,701,54.60
1,Закрыта нами,320,24.92
2,Отклонена,105,8.18
3,На рассмотрении,90,7.01
4,Активна,68,5.30


In [140]:
fig = go.Figure(go.Bar(x=status['статус'], y=status['count'],\
                       text=status['count'].astype('str')+'<br>'+status['ratio_pct'].astype('str')+'%',\
                       marker=dict(color = status.index, colorscale='rainbow')))
fig.update_layout(title_text='Количество вакансий по статусам',
                 width=1000,
                 height=800,
                 title_x=0.5,
                 title_xanchor='center',
                 xaxis_title='Статус вакансии',
                 yaxis_title='Количество вакансий')
fig.show()

Половина вакансий находится в статусе "архив": это значит, что компании не удалось найти подходящую кандидатуру. Закрыты компанией 17% вакансий. Заказчик отказался от рассмотрения пула кандидатов - 10,3%. На рассмотрении находится 8,2% вакансий. Вакансия имеет статус активна - 7,7% от общего количества.

Изучим формат работы. Для этого посмотрим уникальные значения из столбца `формат работы`.

In [141]:
data['формат работы'].sort_values().unique()

array(['Гибрид', 'Гибрид, Удаленка', 'Офис', 'Офис, Гибрид',
       'Офис, Гибрид, Удаленка', 'Офис, Удаленка', 'Удаленка'],
      dtype=object)

Возможные форматы работы: гибрид, офис, удаленка либо их комбинации.

Посмотрим сколько вакансий каждого формата.

In [142]:
job_format = data.groupby('формат работы').agg(count=('формат работы', 'count')).sort_values(by='count', ascending=False)\
.reset_index()
job_format['ratio_pct'] = round(job_format['count'] / len(data) * 100, 2)
job_format

,формат работы,count,ratio_pct
0,Удаленка,652,50.78
1,"Офис, Гибрид",218,16.98
2,"Офис, Гибрид, Удаленка",133,10.36
3,Офис,106,8.26
4,"Гибрид, Удаленка",99,7.71
5,Гибрид,65,5.06
6,"Офис, Удаленка",11,0.86


In [143]:
fig = go.Figure(go.Bar(x=job_format['формат работы'], y=job_format['count'],\
                       text=job_format['count'].astype('str')+'<br>'+job_format['ratio_pct'].astype('str')+'%',\
                       marker=dict(color = job_format.index, colorscale='rainbow')))
fig.update_layout(title_text='Количество вакансий по форматам работы',
                 width=1000,
                 height=800,
                 title_x=0.5,
                 title_xanchor='center',
                 xaxis_title='Формат работы',
                 yaxis_title='Количество вакансий')
fig.show()

Половина вакансий предполагает только удаленный формат работы. Работа либо в офисе, либо гибрид предлагается в 17% вакансий. Любой формат работы представлен в 10% вакансий. Только офис - 8,2%. Гибрид или удаленка - 7,7% вакансий. Формат работы только гибрид у 5% вакансий. Формат работы офис или удаленка составляют менее процента.

Изучим вакансии по формату оформления.

In [144]:
employment_format = data.groupby('формат оформления').agg(count=('формат оформления', 'count'))\
.sort_values(by='count', ascending=False).reset_index()
employment_format['ratio_pct'] = round(employment_format['count'] / len(data) * 100, 2)
employment_format

,формат оформления,count,ratio_pct
0,"ИП, ГПХ, Самозанятость, Контракт, Трудовой договор",484,37.69
1,Трудовой договор,334,26.01
2,Самозанятость,132,10.28
3,Контракт,70,5.45
4,"Самозанятость, ИП",58,4.52
5,"Самозанятость, Контракт, ИП, ГПХ",57,4.44
6,"Самозанятость, Трудовой договор",21,1.64
7,ГПХ,20,1.56
8,"Самозанятость, ИП, ГПХ",15,1.17
9,"Самозанятость, ИП, Трудовой договор",14,1.09


In [145]:
fig = go.Figure(go.Bar(y=employment_format['формат оформления'], x=employment_format['count'], orientation='h',\
                       text=employment_format['count'].astype('str')+'<br>'+employment_format['ratio_pct'].astype('str')+'%',\
                       marker=dict(color = employment_format.index, colorscale='rainbow')))
fig.update_layout(title_text='Количество вакансий по форматам оформления',
                 width=1000,
                 height=1000,
                 title_x=0.5,
                 title_xanchor='center',
                 xaxis_title='Количество вакансий',
                 yaxis_title='Формат оформления')
fig.update_yaxes(categoryorder='total ascending')
fig.show()

Почти 38% вакансий предусматривают любую форму оформления кандидата на работу. Оформление только по трудовому договору предусматривают 26% вакансий, только самозанятым - 10%, только контракт 5,4%. Оставшиеся 20,5% вакансий приходятся на другие виды оформления либо комбинации разных видов оформления.

Изучим требования к опыту кандидатов.

Посмотрим уникальные значения столбца `опыт`.

In [146]:
data['опыт'].sort_values().unique()

array(['Без опыта', 'До 1 года опыта, возможен опыт в pet-проектах',
       'До 1 года опыта, возможен опыт в pet-проектах; без опыта',
       'Опыт от 1 года'], dtype=object)

Всего 4 уникальных значения, варьирующихся от "Без опыта", до "Опыт от 1 года". По видимому в датасете представлены вакансии для начинающих специалистов.

Посмотрим на количество вакансий в разрезе требований к опыту.

In [147]:
work_experience = data.groupby('опыт').agg(count=('опыт', 'count')).sort_values(by='count', ascending=False)\
.reset_index()
work_experience['ratio_pct'] = round(work_experience['count'] / len(data) * 100, 2)
work_experience

,опыт,count,ratio_pct
0,"До 1 года опыта, возможен опыт в pet-проектах; без опыта",1180,91.90
1,"До 1 года опыта, возможен опыт в pet-проектах",76,5.92
2,Опыт от 1 года,26,2.02
3,Без опыта,2,0.16


In [148]:
fig = go.Figure(go.Bar(x=work_experience['опыт'], y=work_experience['count'],\
                       text=work_experience['count'].astype('str')+'<br>'+work_experience['ratio_pct'].astype('str')+'%',\
                       marker=dict(color = work_experience.index, colorscale='rainbow')))
fig.update_layout(title_text='Количество вакансий по требованиям к опыту работы',
                 width=1000,
                 height=800,
                 title_x=0.5,
                 title_xanchor='center',
                 xaxis_title='Требования к опыту работы',
                 yaxis_title='Количество вакансий')
fig.show()

Почти 92% вакансий с требованиями к опыту вида "До 1 года опыта, возможен опыт в pet-проектах; без опыта". Всего 2 вакансии (0,16%) где опыт не требуется вовсе. Вакансии где требуется опыт от 1 года составляют 2% (26 штук) от общего количетсва.

Изучим требования к образованию кандидатов.

Посмотрим уникальные значения столбца `образование`.

In [149]:
data['образование'].sort_values().unique()

array(['Другое, На смежных специальностях, Профильное',
       'На смежных специальностях, Профильное', 'Профильное'],
      dtype=object)

Требования к образованию варьируются от фактически любого до профильного.

Посмотрим на количество вакансий в разрезе требований к образованию.

In [150]:
education = data.groupby('образование').agg(count=('образование', 'count')).sort_values(by='count', ascending=False)\
.reset_index()
education['ratio_pct'] = round(education['count'] / len(data) * 100, 2)
education

,образование,count,ratio_pct
0,"Другое, На смежных специальностях, Профильное",1240,96.57
1,"На смежных специальностях, Профильное",42,3.27
2,Профильное,2,0.16


In [151]:
fig = go.Figure(go.Bar(x=education['образование'], y=education['count'],\
                       text=education['count'].astype('str')+'<br>'+education['ratio_pct'].astype('str')+'%',\
                       marker=dict(color = education.index, colorscale='rainbow')))
fig.update_layout(title_text='Количество вакансий по требованиям к образованию',
                 width=1000,
                 height=800,
                 title_x=0.5,
                 title_xanchor='center',
                 xaxis_title='Требования к образованию',
                 yaxis_title='Количество вакансий')
fig.show()

Требования к образованию "Другое, На смежных специальностях, Профильное" у 96,5% вакансий. И только у двух вакансий (0,16%) требуется профильное образование.

Изучим возможные виды занятости.

Посмотрим уникальные значения столбца `занятость`.

In [152]:
data['занятость'].sort_values().unique()

array(['Волонтерство',
       'Волонтерство, Проектная, Частичная, Стажировка, Полная', 'Полная',
       'Проектная', 'Проектная, Полная', 'Проектная, Частичная',
       'Проектная, Частичная, Полная', 'Стажировка',
       'Стажировка, Волонтерство', 'Стажировка, Волонтерство, Проектная',
       'Стажировка, Волонтерство, Проектная, Частичная',
       'Стажировка, Волонтерство, Частичная', 'Стажировка, Полная',
       'Стажировка, Проектная', 'Стажировка, Проектная, Полная',
       'Стажировка, Проектная, Частичная',
       'Стажировка, Проектная, Частичная, Полная',
       'Стажировка, Частичная', 'Стажировка, Частичная, Полная',
       'Частичная', 'Частичная, Полная'], dtype=object)

Можно выделить пять основных типов занятости: волонтерство, стажировка, проектная, частичная, полная. В датасете эти типы занятости представлкны либо отдельно, либо в различных комбинациях.

Посмотрим на количество вакансий по типам занятости.

In [153]:
employment = data.groupby('занятость').agg(count=('занятость', 'count')).sort_values(by='count', ascending=False)\
.reset_index()
employment['ratio_pct'] = round(employment['count'] / len(data) * 100, 2)
employment

,занятость,count,ratio_pct
0,Полная,774,60.28
1,"Волонтерство, Проектная, Частичная, Стажировка, Полная",97,7.55
2,Стажировка,82,6.39
3,"Проектная, Частичная",59,4.60
4,Частичная,42,3.27
5,Проектная,40,3.12
6,"Проектная, Частичная, Полная",37,2.88
7,"Частичная, Полная",33,2.57
8,"Стажировка, Проектная, Частичная",18,1.40
9,"Стажировка, Проектная",15,1.17


In [154]:
fig = go.Figure(go.Bar(y=employment['занятость'], x=employment['count'], orientation='h',\
                       text=employment['count'].astype('str')+'<br>'+employment['ratio_pct'].astype('str')+'%',\
                       marker=dict(color = employment.index, colorscale='rainbow')))
fig.update_layout(title_text='Количество вакансий по типам занятости',
                 width=1000,
                 height=1000,
                 title_x=0.5,
                 title_xanchor='center',
                 xaxis_title='Количество вакансий',
                 yaxis_title='Тип занятости')
fig.update_yaxes(categoryorder='total ascending')
fig.show()

Большинство вакансий - 60% предполагают полную занятость. Все типы занятости предлагаются у 7,5% вакансий (интересно в данном случае выбор типа занятости остается за кандидатом?). Стажировка предлагается в 82 вакансиях (6,39%). "Проектная, Частичная" занятость подразумевается в 59 вакансиях (4,6%). Только частичная занятость - 42 вакансии (3,27%), только проектная - 40 вакансия (3,12%). Только волонтеров ищут в двух вакансиях (0,16%). Оставшаяся 15% вакансий приходятся на различные комбинации типов занятости.

### Построение гистограмм просмотров и откликов на вакансии

Построим гистограмму количества просмотров вакансий.

In [155]:
fig = go.Figure(go.Histogram(x=data['количество просмотров']))
fig.update_layout(title_text='Гистограмма количества просмотров вакансий',
                 width=1000,
                 height=800,
                 title_x=0.5,
                 title_xanchor='center',
                 xaxis_title='Количество просмотров',
                 yaxis_title='Количество вакансий')
fig.update_traces(marker_line_width=1, selector=dict(type='histogram'))
fig.show()

Построим гистограмму количества откликов на вакансии.

In [156]:
fig = go.Figure(go.Histogram(x=data['количество откликов']))
fig.update_layout(title_text='Гистограмма количества откликов на вакансии',
                 width=1000,
                 height=800,
                 title_x=0.5,
                 title_xanchor='center',
                 xaxis_title='Количество откликов',
                 yaxis_title='Количество вакансий')
fig.update_traces(marker_line_width=1, selector=dict(type='histogram'))
fig.show()

Большинство вакансий имеет до 100 просмотров и до 30 откликов.

### Добавление новых признаков (столбцов) в дафарейм для последующей проверки гипотез.

Добавим в датафрейм столбец `москва` в котором 1 - работа предлагается в Москве, 0 - в других городах.

In [157]:
data['москва'] = data['город'].map(lambda x: 1 if 'Москва' in x else 0)

Добавим в датафрейм столбец `зарплата` в котором 1 - указана зарплата, 0 - не указана зарплата.

In [158]:
data['зарплата'] = data.apply(lambda x: 1 if x['зарплата от'] >= 0 or x['зарплата до'] >= 0 else 0, axis=1)

Добавим в датафрейм столбец `удаленка` в котором 1 - вакансии с удаленной работой, 0 - вакансии с посещением офиса.

In [159]:
data['удаленка'] = data['формат работы'].map(lambda x: 1 if 'Удаленка' in x else 0)

Добавим в датафрейм столбец `полная занятость` в котором 1 - вакансии с полной занятостью, 0 - вакансии с другими видами занятости.

In [160]:
data['полная занятость'] = data['занятость'].map(lambda x: 1 if 'Полная' in x else 0)

На этапе исследовательского анализа данных мы изучили вакансии по:
- статусам и выяснили, что половина вакансий находится в статусе "архив": это значит, что компании не удалось найти подходящую кандидатуру. Закрыты компанией 17% вакансий. Заказчик отказался от рассмотрения пула кандидатов - 10,3%. На рассмотрении находится 8,2% вакансий. Вакансия имеет статус активна - 7,7% от общего количества;
- форматам работы и выяснили, что половина вакансий предполагает только удаленный формат работы. Работа либо в офисе, либо гибрид предлагается в 17% вакансий. Любой формат работы представлен в 10% вакансий. Только офис - 8,2%. Гибрид или удаленка - 7,7% вакансий. Формат работы только гибрид у 5% вакансий. Формат работы офис или удаленка составляют менее процента;
- форматам оформления и выяснили, что почти 38% вакансий предусматривают любую форму оформления кандидата на работу. Оформление только по трудовому договору предусматривают 26% вакансий, только самозанятым - 10%, только контракт 5,4%. Оставшиеся 20,5% вакансий приходятся на другие виды оформления либо комбинации разных видов оформления;
- требованиям к опыту кандидатов и выяснили, что почти 92% вакансий с требованиями к опыту вида "До 1 года опыта, возможен опыт в pet-проектах; без опыта". Всего 2 вакансии (0,16%) где опыт не требуется вовсе. Вакансии где требуется опыт от 1 года составляют 2% (26 штук) от общего количетсва;
- требованиям к образованию кандидатов и выяснили, что требования к образованию "Другое, На смежных специальностях, Профильное" у 96,5% вакансий. И только у двух вакансий (0,16%) требуется профильное образование;
- возможным видам занятости и выяснили, что большинство вакансий - 60% предполагают полную занятость. Все типы занятости предлагаются у 7,5% вакансий. Стажировка предлагается в 82 вакансиях (6,39%). "Проектная, Частичная" занятость подразумевается в 59 вакансиях (4,6%). Только частичная занятость - 42 вакансии (3,27%), только проектная - 40 вакансия (3,12%). Только волонтеров ищут в двух вакансиях (0,16%). Оставшаяся 15% вакансий приходятся на различные комбинации типов занятости.

Построили гистограммы количества просмотров и откликов на вакансии: большинство вакансий имеет до 100 просмотров и до 30 откликов. Для проверки гипотез добавили в дафарейм новые признаки.

<a id='4'></a>
## [Проверка гипотез](#6)

### Гипотеза № 1. Работу в Москве ищет больше людей, чем работу в других городах.

**Нулевая гипотеза H0:** Среднее количество просмотров вакансий с работой в Москве и других городах одинаковое.<br>
**Альтернативная гипотеза H1:** Среднее количество просмотров вакансий с работой в Москве больше, чем в других городах.<br>
Уровень значимости aplha = 0.05.

In [161]:
alpha = 0.05
result = st.ttest_ind(data[data['москва'] == 1]['количество просмотров'], data[data['москва'] == 0]['количество просмотров'],\
                      alternative='greater')
print('Среднее количество просмотров вакансий с работой в Москве:',\
      round(data[data['москва'] == 1]['количество просмотров'].mean(), 2))
print('Среднее количество просмотров вакансий с работой в других городах:',\
      round(data[data['москва'] == 0]['количество просмотров'].mean(), 2))
print('p-значение:', result.pvalue)
if result.pvalue < alpha:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, есть основания отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве просмотров вакансий с работой в Москве и других городах в пользу альтернативной гипотезы о '
          'большем среднем количестве просмотров вакансий с работой в Москве, чем в других городах.')
else:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, нет оснований отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве просмотров вакансий с работой в Москве и других городах в пользу альтернативной гипотезы о '
          'большем среднем количестве просмотров вакансий с работой в Москве, чем в других городах.')

Среднее количество просмотров вакансий с работой в Москве: 40.26
Среднее количество просмотров вакансий с работой в других городах: 40.0
p-значение: 0.4590014971684584
По имеющимся данным, на уровне значимости 5%, нет оснований отвергнуть нулевую гипотезу об отсутствии различий в среднем 
количестве просмотров вакансий с работой в Москве и других городах в пользу альтернативной гипотезы о большем среднем количестве просмотров вакансий с работой в Москве, чем в других городах.


**Нулевая гипотеза H0:** Среднее количество откликов на вакансии с работой в Москве и других городах одинаковое.<br>
**Альтернативная гипотеза H1:** Среднее количество откликов на вакансии с работой в Москве больше, чем в других городах.<br>
Уровень значимости aplha = 0.05.

In [162]:
alpha = 0.05
result = st.ttest_ind(data[data['москва'] == 1]['количество откликов'], data[data['москва'] == 0]['количество откликов'],\
                      alternative='greater')
print('Среднее количество откликов на вакансии с работой в Москве:',\
      round(data[data['москва'] == 1]['количество откликов'].mean(), 2))
print('Среднее количество откликов на вакансии с работой в других городах:',\
      round(data[data['москва'] == 0]['количество откликов'].mean(), 2))
print('p-значение:', result.pvalue)
if result.pvalue < alpha:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, есть основания отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве откликов на вакансии с работой в Москве и других городах в пользу альтернативной гипотезы о '
          'большем среднем \nколичестве откликов на вакансии с работой в Москве, чем в других городах.')
else:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, нет оснований отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве откликов на вакансии с работой в Москве и других городах в пользу альтернативной гипотезы о '
          'большем среднем \nколичестве откликов на вакансии с работой в Москве, чем в других городах.')

Среднее количество откликов на вакансии с работой в Москве: 11.53
Среднее количество откликов на вакансии с работой в других городах: 11.47
p-значение: 0.46957763343322434
По имеющимся данным, на уровне значимости 5%, нет оснований отвергнуть нулевую гипотезу об отсутствии различий в среднем 
количестве откликов на вакансии с работой в Москве и других городах в пользу альтернативной гипотезы о большем среднем 
количестве откликов на вакансии с работой в Москве, чем в других городах.


Гипотеза опровергнута. Люди одинаково активно ищут работу везде: и в Москве и других городах.

### Гипотеза № 2. Люди активнее просматривают и откликаются на вакансии в которых указан размер заработной платы

**Нулевая гипотеза H0:** Среднее количество просмотров вакансий с указанной зарплатой и вакансий где зарплата не указана одинаковое.<br>
**Альтернативная гипотеза H1:** Среднее количество просмотров вакансий с указанной зарплатой больше, чем вакансий где зарплата не указана.<br>
Уровень значимости aplha = 0.05.

In [163]:
alpha = 0.05
result = st.ttest_ind(data[data['зарплата'] == 1]['количество просмотров'], data[data['зарплата'] == 0]['количество просмотров'],\
                      alternative='greater')
print('Среднее количество просмотров вакансий с указанной зарплатой:',\
      round(data[data['зарплата'] == 1]['количество просмотров'].mean(), 2))
print('Среднее количество просмотров вакансий где зарплата не указана:',\
      round(data[data['зарплата'] == 0]['количество просмотров'].mean(), 2))
print('p-значение:', result.pvalue)
if result.pvalue < alpha:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, есть основания отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве просмотров вакансий с указанной зарплатой и вакансий где зарплата не указана в пользу '
          'альтернативной гипотезы о \nбольшем среднем количестве просмотров вакансий с указанной зарплатой, чем вакансий '
          'где зарплата не указана.')
else:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, нет оснований отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве просмотров вакансий с указанной зарплатой и вакансий где зарплата не указана в пользу '
          'альтернативной гипотезы о \nбольшем среднем количестве просмотров вакансий с указанной зарплатой, чем вакансий '
          'где зарплата не указана.')

Среднее количество просмотров вакансий с указанной зарплатой: 39.87
Среднее количество просмотров вакансий где зарплата не указана: 40.42
p-значение: 0.5991107595121397
По имеющимся данным, на уровне значимости 5%, нет оснований отвергнуть нулевую гипотезу об отсутствии различий в среднем 
количестве просмотров вакансий с указанной зарплатой и вакансий где зарплата не указана в пользу альтернативной гипотезы о 
большем среднем количестве просмотров вакансий с указанной зарплатой, чем вакансий где зарплата не указана.


**Нулевая гипотеза H0:** Среднее количество откликов на вакансии с указанной зарплатой и вакансии где зарплата не указана одинаковое.<br>
**Альтернативная гипотеза H1:** Среднее количество откликов на вакансии с указанной зарплатой больше, чем на вакансии где зарплата не указана.<br>
Уровень значимости aplha = 0.05.

In [164]:
alpha = 0.05
result = st.ttest_ind(data[data['зарплата'] == 1]['количество откликов'], data[data['зарплата'] == 0]['количество откликов'],\
                      alternative='greater')
print('Среднее количество откликов на вакансии с указанной зарплатой:',\
      round(data[data['зарплата'] == 1]['количество откликов'].mean(), 2))
print('Среднее количество откликов на вакансии где зарплата не указана:',\
      round(data[data['зарплата'] == 0]['количество откликов'].mean(), 2))
print('p-значение:', result.pvalue)
if result.pvalue < alpha:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, есть основания отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве откликов на вакансии с указанной зарплатой и вакансии где зарплата не указана в пользу '
          'альтернативной гипотезы о \nбольшем среднем количестве откликов на вакансии с указанной зарплатой, чем на вакансии '
          'где зарплата не указана.')
else:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, нет оснований отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве откликов на вакансии с указанной зарплатой и вакансии где зарплата не указана в пользу '
          'альтернативной гипотезы о \nбольшем среднем количестве откликов на вакансии с указанной зарплатой, чем на вакансии '
          'где зарплата не указана.')

Среднее количество откликов на вакансии с указанной зарплатой: 11.45
Среднее количество откликов на вакансии где зарплата не указана: 11.56
p-значение: 0.5598113907524338
По имеющимся данным, на уровне значимости 5%, нет оснований отвергнуть нулевую гипотезу об отсутствии различий в среднем 
количестве откликов на вакансии с указанной зарплатой и вакансии где зарплата не указана в пользу альтернативной гипотезы о 
большем среднем количестве откликов на вакансии с указанной зарплатой, чем на вакансии где зарплата не указана.


Гипотеза опровергнута. Людям похоже все равно какая зарплата, лишь бы работу найти 🙂.

### Гипотеза № 3. Люди активнее просматривают и откликаются на вакансии с удаленной работой, чем на вакансии, которые подразумевают поездки в офис

**Нулевая гипотеза H0:** Среднее количество просмотров вакансий с удаленной работой и вакансий с посещением офиса одинаковое.<br>
**Альтернативная гипотеза H1:** Среднее количество просмотров вакансий с удаленной работой больше, чем вакансий с посещением офиса.<br>
Уровень значимости aplha = 0.05.

In [165]:
alpha = 0.05
result = st.ttest_ind(data[data['удаленка'] == 1]['количество просмотров'], data[data['удаленка'] == 0]['количество просмотров'],\
                      alternative='greater')
print('Среднее количество просмотров вакансий с удаленной работой:',\
      round(data[data['удаленка'] == 1]['количество просмотров'].mean(), 2))
print('Среднее количество просмотров вакансий с посещением офиса:',\
      round(data[data['удаленка'] == 0]['количество просмотров'].mean(), 2))
print('p-значение:', result.pvalue)
if result.pvalue < alpha:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, есть основания отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве просмотров вакансий с удаленной работой и вакансий с посещением офиса в пользу '
          'альтернативной гипотезы о большем \nсреднем количестве просмотров вакансий с удаленной работой, чем вакансий '
          'с посещением офиса.')
else:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, нет оснований отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве просмотров вакансий с удаленной работой и вакансий с посещением офиса в пользу '
          'альтернативной гипотезы о большем \nсреднем количестве просмотров вакансий с удаленной работой, чем вакансий '
          'с посещением офиса.')

Среднее количество просмотров вакансий с удаленной работой: 46.86
Среднее количество просмотров вакансий с посещением офиса: 24.86
p-значение: 9.074170272249666e-22
По имеющимся данным, на уровне значимости 5%, есть основания отвергнуть нулевую гипотезу об отсутствии различий в среднем 
количестве просмотров вакансий с удаленной работой и вакансий с посещением офиса в пользу альтернативной гипотезы о большем 
среднем количестве просмотров вакансий с удаленной работой, чем вакансий с посещением офиса.


**Нулевая гипотеза H0:** Среднее количество откликов на вакансии с удаленной работой и вакансии с посещением офиса одинаковое.<br>
**Альтернативная гипотеза H1:** Среднее количество откликов на вакансии с удаленной работой больше, чем на вакансии с посещением офиса.<br>
Уровень значимости aplha = 0.05.

In [166]:
alpha = 0.05
result = st.ttest_ind(data[data['удаленка'] == 1]['количество откликов'], data[data['удаленка'] == 0]['количество откликов'],\
                      alternative='greater')
print('Среднее количество откликов на вакансии с удаленной работой:',\
      round(data[data['удаленка'] == 1]['количество откликов'].mean(), 2))
print('Среднее количество откликов на вакансии с посещением офиса:',\
      round(data[data['удаленка'] == 0]['количество откликов'].mean(), 2))
print('p-значение:', result.pvalue)
if result.pvalue < alpha:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, есть основания отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве откликов на вакансии с удаленной работой и вакансии с посещением офиса в пользу '
          'альтернативной гипотезы о большем \nсреднем количестве откликов на вакансии с удаленной работой, чем на вакансии '
          'с посещением офиса.')
else:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, нет оснований отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве откликов на вакансии с удаленной работой и вакансии с посещением офиса в пользу '
          'альтернативной гипотезы о большем \nсреднем количестве откликов на вакансии с удаленной работой, чем на вакансии '
          'с посещением офиса.')

Среднее количество откликов на вакансии с удаленной работой: 13.14
Среднее количество откликов на вакансии с посещением офиса: 7.78
p-значение: 1.033891744576152e-12
По имеющимся данным, на уровне значимости 5%, есть основания отвергнуть нулевую гипотезу об отсутствии различий в среднем 
количестве откликов на вакансии с удаленной работой и вакансии с посещением офиса в пользу альтернативной гипотезы о большем 
среднем количестве откликов на вакансии с удаленной работой, чем на вакансии с посещением офиса.


Гипотеза подтверждена. В наше время люди предпочитают работать удаленно.

### Гипотеза № 4. Люди активнее просматривают и откликаются на вакансии с полной занятостью, чем на вакансии с другими видами занятости.

**Нулевая гипотеза H0:** Среднее количество просмотров вакансий с полной занятостью и вакансий с другими видами занятости одинаковое.<br>
**Альтернативная гипотеза H1:** Среднее количество просмотров вакансий с полной занятостью меньше, чем вакансий с другими видами занятости.<br>
Уровень значимости aplha = 0.05.

In [167]:
alpha = 0.05
result = st.ttest_ind(data[data['полная занятость'] == 1]['количество просмотров'],\
                      data[data['полная занятость'] == 0]['количество просмотров'],\
                      alternative='less')
print('Среднее количество просмотров вакансий с полной занятостью:',\
      round(data[data['полная занятость'] == 1]['количество просмотров'].mean(), 2))
print('Среднее количество просмотров вакансий с другими видами занятости:',\
      round(data[data['полная занятость'] == 0]['количество просмотров'].mean(), 2))
print('p-значение:', result.pvalue)
if result.pvalue < alpha:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, есть основания отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве просмотров вакансий с полной занятостью и вакансий с другими видами занятости в пользу '
          'альтернативной гипотезы о \nбольшем среднем количестве просмотров вакансий с полной занятостью, чем вакансий '
          'с другими видами занятости.')
else:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, нет оснований отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве просмотров вакансий с полной занятостью и вакансий с другими видами занятости в пользу '
          'альтернативной гипотезы о \nбольшем среднем количестве просмотров вакансий с полной занятостью, чем вакансий '
          'с другими видами занятости.')

Среднее количество просмотров вакансий с полной занятостью: 37.1
Среднее количество просмотров вакансий с другими видами занятости: 50.39
p-значение: 8.591235754497523e-08
По имеющимся данным, на уровне значимости 5%, есть основания отвергнуть нулевую гипотезу об отсутствии различий в среднем 
количестве просмотров вакансий с полной занятостью и вакансий с другими видами занятости в пользу альтернативной гипотезы о 
большем среднем количестве просмотров вакансий с полной занятостью, чем вакансий с другими видами занятости.


**Нулевая гипотеза H0:** Среднее количество откликов на вакансии с полной занятостью и вакансии с другими видами занятости одинаковое.<br>
**Альтернативная гипотеза H1:** Среднее количество откликов на вакансии с полной занятостью меньше, чем на вакансии с другими видами занятости.<br>
Уровень значимости aplha = 0.05.

In [168]:
alpha = 0.05
result = st.ttest_ind(data[data['полная занятость'] == 1]['количество откликов'],\
                      data[data['полная занятость'] == 0]['количество откликов'],\
                      alternative='less')
print('Среднее количество откликов на вакансии с полной занятостью:',\
      round(data[data['полная занятость'] == 1]['количество откликов'].mean(), 2))
print('Среднее количество откликов на вакансии с другими видами занятости:',\
      round(data[data['полная занятость'] == 0]['количество откликов'].mean(), 2))
print('p-значение:', result.pvalue)
if result.pvalue < alpha:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, есть основания отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве откликов на вакансии с полной занятостью и вакансии с другими видами занятости в пользу '
          'альтернативной гипотезы о \nменьшем среднем количестве откликов на вакансии с полной занятостью, чем на вакансии '
          'с другими видами занятости.')
else:
    print('По имеющимся данным, на уровне значимости {0:.0f}%, нет оснований отвергнуть нулевую гипотезу об отсутствии различий'\
                  .format(alpha * 100),
          'в среднем \nколичестве откликов на вакансии с полной занятостью и вакансии с другими видами занятости в пользу '
          'альтернативной гипотезы о \nменьшем среднем количестве откликов на вакансии с полной занятостью, чем на вакансии '
          'с другими видами занятости.')

Среднее количество откликов на вакансии с полной занятостью: 11.18
Среднее количество откликов на вакансии с другими видами занятости: 12.6
p-значение: 0.04565163494934621
По имеющимся данным, на уровне значимости 5%, есть основания отвергнуть нулевую гипотезу об отсутствии различий в среднем 
количестве откликов на вакансии с полной занятостью и вакансии с другими видами занятости в пользу альтернативной гипотезы о 
меньшем среднем количестве откликов на вакансии с полной занятостью, чем на вакансии с другими видами занятости.


Гипотеза опровергнута. Хотя вакансий с полной занятостью гораздо больше, чем вакансий с другими видами занятости, люди тем не менее в среднем гораздо чаще просматривают вакансии с другими видами занятости и чаще на них откликаются пусть и не намного.

На этапе проверки гипотез мы проверили 4 гипотезы:
- **гипотеза № 1. Работу в Москве ищет больше людей, чем работу в других городах:**
     * гипотеза опровергнута. Люди одинаково активно ищут работу везде: и в Москве и других городах;
- **гипотеза № 2. Люди активнее просматривают и откликаются на вакансии в которых указан размер заработной платы:**
     * гипотеза опровергнута. Люди одинаково активно ищут не смотря на то, указана зарплата в вакансии или нет;
- **гипотеза № 3. Люди активнее просматривают и откликаются на вакансии с удаленной работой, чем на вакансии, которые подразумевают поездки в офис:**
     * гипотеза подтверждена. В наше время люди предпочитают работать удаленно;
- **гипотеза № 4. Люди активнее просматривают и откликаются на вакансии с полной занятостью, чем на вакансии с другими видами занятости:**
     * гипотеза опровергнута. Хотя вакансий с полной занятостью гораздо больше, чем вакансий с другими видами занятости, люди тем не менее в среднем гораздо чаще просматривают вакансии с другими видами занятости и чаще на них откликаются пусть и не намного.

<a id='5'></a>
## [Общий вывод](#6)

**Этап ознакомления с данными**

В датафрейме 1284 строк и 22 столбца. Имеется достаточное количетсво данных для проведения исследования. В датафрейме есть пропущеные значения которые заполняются опционально. Столбцы `Дата публикации` и `Дата закрытия` нужно привести в формат даты.

**Этап предобработки данных**

На этапе предобработки данных мы переименовали столбцы в таблице, чтобы они начинались с маленькой буквы, изменили тип столбцов `дата публикации` и `дата закрытия` в датафрейме на datetime, убедились в отсутствии явных дубликатов в датафреймах.

**Этап исследовательского анализа данных**

На этапе исследовательского анализа данных мы изучили вакансии по:
- статусам и выяснили, что половина вакансий находится в статусе "архив": это значит, что компании не удалось найти подходящую кандидатуру. Закрыты компанией 17% вакансий. Заказчик отказался от рассмотрения пула кандидатов - 10,3%. На рассмотрении находится 8,2% вакансий. Вакансия имеет статус активна - 7,7% от общего количества;
- форматам работы и выяснили, что половина вакансий предполагает только удаленный формат работы. Работа либо в офисе, либо гибрид предлагается в 17% вакансий. Любой формат работы представлен в 10% вакансий. Только офис - 8,2%. Гибрид или удаленка - 7,7% вакансий. Формат работы только гибрид у 5% вакансий. Формат работы офис или удаленка составляют менее процента;
- форматам оформления и выяснили, что почти 38% вакансий предусматривают любую форму оформления кандидата на работу. Оформление только по трудовому договору предусматривают 26% вакансий, только самозанятым - 10%, только контракт 5,4%. Оставшиеся 20,5% вакансий приходятся на другие виды оформления либо комбинации разных видов оформления;
- требованиям к опыту кандидатов и выяснили, что почти 92% вакансий с требованиями к опыту вида "До 1 года опыта, возможен опыт в pet-проектах; без опыта". Всего 2 вакансии (0,16%) где опыт не требуется вовсе. Вакансии где требуется опыт от 1 года составляют 2% (26 штук) от общего количетсва;
- требованиям к образованию кандидатов и выяснили, что требования к образованию "Другое, На смежных специальностях, Профильное" у 96,5% вакансий. И только у двух вакансий (0,16%) требуется профильное образование;
- возможным видам занятости и выяснили, что большинство вакансий - 60% предполагают полную занятость. Все типы занятости предлагаются у 7,5% вакансий. Стажировка предлагается в 82 вакансиях (6,39%). "Проектная, Частичная" занятость подразумевается в 59 вакансиях (4,6%). Только частичная занятость - 42 вакансии (3,27%), только проектная - 40 вакансия (3,12%). Только волонтеров ищут в двух вакансиях (0,16%). Оставшаяся 15% вакансий приходятся на различные комбинации типов занятости.

Построили гистограммы количества просмотров и откликов на вакансии: большинство вакансий имеет до 100 просмотров и до 30 откликов. Для проверки гипотез добавили в дафарейм новые признаки.

**Этап проверки гипотез**

На этапе проверки гипотез мы проверили 4 гипотезы:
- **гипотеза № 1. Работу в Москве ищет больше людей, чем работу в других городах:**
     * гипотеза опровергнута. Люди одинаково активно ищут работу везде: и в Москве и других городах;
- **гипотеза № 2. Люди активнее просматривают и откликаются на вакансии в которых указан размер заработной платы:**
     * гипотеза опровергнута. Люди одинаково активно ищут не смотря на то, указана зарплата в вакансии или нет;
- **гипотеза № 3. Люди активнее просматривают и откликаются на вакансии с удаленной работой, чем на вакансии, которые подразумевают поездки в офис:**
     * гипотеза подтверждена. В наше время люди предпочитают работать удаленно;
- **гипотеза № 4. Люди активнее просматривают и откликаются на вакансии с полной занятостью, чем на вакансии с другими видами занятости:**
     * гипотеза опровергнута. Хотя вакансий с полной занятостью гораздо больше, чем вакансий с другими видами занятости, люди тем не менее в среднем гораздо чаще просматривают вакансии с другими видами занятости и чаще на них откликаются пусть и не намного.